# Getting Albums for All Rappers

In [6]:
import requests
import os
from dotenv import load_dotenv
from utils import Album, remove_duplicates

Use environment variables to get Spotify OAuth token and set up request headers.

In [7]:
load_dotenv('../.env')
SPOTIFY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIFY_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')
SPOTIFY_AUTH_URL = 'https://accounts.spotify.com/api/token'

spotify_auth_reponse = requests.post(SPOTIFY_AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': SPOTIFY_CLIENT_ID,
    'client_secret': SPOTIFY_CLIENT_SECRET
})

auth_data = spotify_auth_reponse.json()
access_token = auth_data['access_token']
request_headers = {
    'Authorization': 'Bearer {}'.format(access_token)
}

Get list of rappers from `rappers.txt`.

In [8]:
with open('rappers.txt', 'r') as rappers_file:
    rappers_list = rappers_file.readlines()
    rappers_list = [rapper.strip() for rapper in rappers_list]

Get rapper ID on Spotify for each rapper.

In [9]:
SPOTIFY_SEARCH_URL = 'https://api.spotify.com/v1/search'

rapper_ids = []
for rapper in rappers_list:
    result = requests.get(SPOTIFY_SEARCH_URL, params={
        'q': rapper,
        'type': 'artist'
    }, headers=request_headers).json()

    rapper_info = result['artists']['items'][0]
    rapper_ids.append(rapper_info['id'])

Get all albums for each rapper and save them in a dictionary to take care of duplicates (e.g. collaborative albums).

In [11]:
SPOTIFY_ALBUMS_URL = 'https://api.spotify.com/v1/artists/{id}/albums?offset=0&limit=50&include_groups=album'
SPOTIFY_GET_ALBUM_URL = 'https://api.spotify.com/v1/albums/{id}'

albums_map = {}
for rapper_id in rapper_ids:
    result = requests.get(SPOTIFY_ALBUMS_URL.format(id=rapper_id), headers=request_headers).json()
    albums = [a for a in result['items'] if a['album_type'] == 'album']
    for a in albums:
        if (not a['id'] in albums_map):
            album = requests.get(SPOTIFY_GET_ALBUM_URL.format(id=a['id']), headers=request_headers).json()
            tracks = album['tracks']['items']
            album_is_explicit = any([track['explicit'] for track in tracks])
            albums_map[a['id']] = Album(a['id'], a['name'], album_is_explicit)

Save all albums in `all_albums.csv` file

In [13]:
all_albums_list = list(albums_map.values())
with open('all_albums.csv', 'w') as all_albums_file:
    all_albums_file.write('Spotify Album ID,Album Name\n')
    for album in all_albums_list:
        all_albums_file.write('{},{}\n'.format(str(album.id), album.name))

Remove clean versions of albums and write them all to `albums.csv`

In [14]:
final_albums_list = remove_duplicates(list(albums_map.values()))
with open('albums.csv', 'w') as albums_file:
    albums_file.write('Spotify Album ID,Album Name\n')
    for album in final_albums_list:
        albums_file.write('{},{}\n'.format(str(album.id), album.name))